In [1]:
import re

with open("GrammarCopy.txt", "r", encoding='utf-8') as f:
    rows = f.readlines()

# убираем пустые строки
for i in range(len(rows) - 1, -1, -1):  # Перебираем в обратном порядке
    row = rows[i]
    words = row.split(' ')
    
    if words[0] == '\n':
        del rows[i]

state_dict = {}

# убираем "направляющие" и прописываем правила
rule = 1
for i in range(len(rows)):
    row = rows[i]
    left = row.split(' ')[0]
    result = re.sub(r'\[.*?\] ', '', row)
    words = result.split(' ')
    
    new_row = str(rule) + ' ' + ' '.join(words)
    rows[i] = new_row

    if left not in state_dict:
        state_dict[left] = rule

    rule += 1

# записываем все состояния
for i, row in enumerate(rows):
    splited = row.split(' ')
    new_row = ''
    rules = splited[3:]
    for current_rule in rules:
        current_rule = ' ' + str(rule) + ' ' + current_rule
        rule += 1
        new_row += current_rule
    
    rows[i] = ' '.join(splited[0:3]) + new_row

with open('newGrammar.txt', 'w', encoding='utf-8') as f:
    f.writelines(rows)

sorted_by_left = list(sorted(rows, key=lambda row: row.split(' ')[1]))
first = sorted_by_left[0].split(' ')[1]

# error создаём
state_table = {i : {'error' : True, 'stack': False, 'return': False, 'accept': False, 'guid_symbols' : []} for i in range(1, rule)}
for row in sorted_by_left:
    splited = row.split(' ')
    state = int(splited[0])
    current = splited[1]
    if first == current:
        state_table[state]['error'] = 0
    else: 
        first = current


# стэк
terminals = ['NUM', 'id', '(', ')', 'true', 'false', 'int', 'void', 'bool', '+', '-', '*', '/', '||', '&&', '<', '>', '=', '<=', '>=', '!=', '==', 'if', 'while', '{', '}', ',', 'else', 'return']
eps = '/eps'
for row in rows:
    splited = row.split(' ')
    right = splited[3:]
    for i in range(0, len(right) - 2, 2):
        state = int(right[i])
        symbol = right[i + 1].replace('\n', '')
        if symbol not in terminals and symbol != eps:
            state_table[state]['stack'] = True

# return
for row in rows:
    splited = row.split(' ')
    last_state = int(splited[-2])
    last_symbol = splited[-1].replace('\n', '')
    if last_symbol in terminals or last_symbol == eps:
        state_table[last_state]['return'] = True

# accept
for row in rows:
    splited = row.split(' ')
    right = splited[3:]
    for i in range(0, len(right), 2):
        state = int(right[i])
        symbol = right[i + 1].replace('\n', '')
        if symbol in terminals or symbol == eps:
            state_table[state]['accept'] = True

# next_state
for row in rows:
    splited = row.split(' ')
    left_state, left_symbol = int(splited[0]), splited[1]
    right = splited[3:]
    state_table[left_state]['next'] = int(right[0])
    for i in range(0, len(right) - 2, 2):
        state = int(right[i])
        symbol = right[i + 1]
        if symbol in terminals or symbol == eps:
            state_table[state]['next_state'] = state + 1
        else:
            state_table[state]['next_state'] = state_dict[symbol]

    last_state, last_symbol = int(right[-2]), right[-1].replace('\n', '')
    if last_symbol not in terminals and last_symbol != eps:
        state_table[last_state]['next_state'] = state_dict[last_symbol]

state_table

{1: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 71},
 2: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 72},
 3: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 75},
 4: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 78},
 5: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 81},
 6: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 82},
 7: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 83},
 8: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 84},
 9: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 88}

In [2]:
# терминальные заполняю
for row in rows:
    splited = row.split(' ')
    left_state, left_symbol = int(splited[0]), splited[1]
    right = splited[3:]

    for i in range(0, len(right), 2):
        state = int(right[i])
        symbol = right[i + 1].replace('\n', '')
        if symbol in terminals:
            state_table[state]['guid_symbols'] = [symbol]

In [3]:
new_state_dict = {}
leftSymbol_leftState = {}

# Для левого состояния получаю список правых
def getRightPartStates(row: str):
    splited = row.split(' ')
    left_state = int(splited[0])
    right = splited[3:]
    right_states = [int(right[i]) for i in range(0, len(right), 2)]
    new_state_dict[left_state] = right_states.copy()

def createDictLEFT_SYMBOL_LEFT_STATE(rows: list[str]):
    for row in rows:
        splited = row.split(' ')
        left_state, left_symbol = int(splited[0]), splited[1]
        if left_symbol in leftSymbol_leftState.keys():
            leftSymbol_leftState[left_symbol].append(left_state)
        else:
            leftSymbol_leftState[left_symbol] = [left_state]

# Все ли направляющие правые заполнены
def isAllGuidSymbolsAdded(symbol: int):
    left_states = leftSymbol_leftState[symbol]
    for state in left_states:
        if len(state_table[state]['guid_symbols']) == 0:
            return False

    return True

for row in rows:
    getRightPartStates(row)

createDictLEFT_SYMBOL_LEFT_STATE(rows)

In [4]:
def getAllGuidSymbolsBySymbol(symbol: str):
    symbols = []
    for row in rows:
        splited = row.split(' ')
        left_state = int(splited[0])
        left_symbol = splited[1]
        if symbol == left_symbol:
            symbols.append(state_table[left_state]['guid_symbols'])

    flat_list = [
        x
        for xs in symbols
        for x in xs
    ]
    return flat_list

getAllGuidSymbolsBySymbol('ОператорСравнения')

[]

In [5]:
leftSymbol_leftState

{'S': [1],
 'ОбъявлениеСтруктуры': [2, 3, 4],
 'ТипСтруктуры': [5, 6, 7],
 'Структура': [8, 9, 10],
 'ПараметрыФункции': [11, 12],
 'ПараметрФункции': [13, 14],
 'СписокПараметровФункции': [15],
 'ИнициализацияПеременной': [16, 17],
 'СоставнойОператор': [18],
 'СписокОператоров': [19, 20],
 'Оператор': [21, 22, 23, 24, 25, 26, 27],
 'СтруктураНачинающеесяСID': [28, 29, 30, 31, 32, 33, 34, 35, 36],
 'ID': [37],
 'ELSE': [38, 39],
 'ВыражениеИлиПусто': [40, 41],
 'Выражение': [42, 43],
 'СписокЛогическихУсловийAND': [44, 45],
 'ЛогическоеУсловиеAND': [46],
 'СписокЛогическихУсловийOR': [47, 48],
 'ЛогическоеУсловниеOR': [49],
 'СписокЛогическихСравнений': [50, 51],
 'ЛогическоеСравнение': [52],
 'СписокСлагаемых': [53, 54, 55],
 'Слагаемое': [56],
 'СписокМножителей': [57, 58, 59],
 'Множитель': [60, 61, 62, 63, 64],
 'ОператорСравнения': [65, 66, 67, 68, 69, 70]}

In [6]:
from copy import deepcopy

# Если терминал справа, то лево заполняем
for row in rows:
    splited = row.split(' ')
    left_state, left_symbol = int(splited[0]), splited[1]
    first_right_state = int(splited[3])
    first_right_symbol = splited[4].replace('\n', '')
    if first_right_symbol in terminals:
        state_table[left_state]['guid_symbols'] = state_table[first_right_state]['guid_symbols'].copy()


# Если первый справа заполнен, заполняем лево
previous_state_table = {}
while previous_state_table != state_table:
    previous_state_table = deepcopy(state_table)
    for row in rows:
        splited = row.split(' ')
        left_state, left_symbol = int(splited[0]), splited[1]
        first_right_state = int(splited[3])
        first_right_symbol = splited[4].replace('\n', '')
        if len(state_table[first_right_state]['guid_symbols']) == 0 and first_right_symbol not in terminals and first_right_symbol != eps:
            if isAllGuidSymbolsAdded(first_right_symbol):
                all_guid_symbols = getAllGuidSymbolsBySymbol(first_right_symbol)
                state_table[first_right_state]['guid_symbols'] = all_guid_symbols.copy()
                state_table[left_state]['guid_symbols'] = state_table[first_right_state]['guid_symbols'].copy()

count = 0
for state, v in state_table.items():
    if len(v['guid_symbols']) != 0:
        count += 1

# state_table
print(count)
print(rule - count) # 109 осталось заполнить

120
93


In [7]:
def processEpsRule(current_row: str):
    current_splited = current_row.split(' ')
    left_state, left_symbol = int(current_splited[0]), current_splited[1]
    symbol = eps
    state = int(current_splited[3])

    for row in rows:
        splited = row.split(' ')
        first_symbol = splited[1]
        last_symbol = splited[-1].replace('\n', '')
        if last_symbol == left_symbol and last_symbol != first_symbol:
            for row2 in rows:
                splited = row2.split(' ')
                splited[-1] = splited[-1].replace('\n', '')
                right = splited[3:]
                # print(right)
                for i in range(0, len(right), 2):
                    this_symbol = right[i + 1]
                    if first_symbol == this_symbol:
                        if i != len(right) - 2:
                            this_state = right[i + 2]
                            this_guid_symbols = getAllGuidSymbolsBySymbol(this_symbol).copy()
                            for guid_symbol in this_guid_symbols:
                                if guid_symbol not in state_table[state]['guid_symbols']:
                                    state_table[state]['guid_symbols'].append(guid_symbol)
                            # state_table[state]['guid_symbols'] += this_guid_symbols.copy()
                        else:
                            if 'END' not in state_table[state]['guid_symbols']:
                                state_table[state]['guid_symbols'].append('END')

main_state_table = {}
while main_state_table != previous_state_table:
    previous_state_table = {'1': -1}
    while previous_state_table != state_table:
        previous_state_table = deepcopy(state_table)
        for row in rows:
            splited = row.split(' ')
            left_state, left_symbol = int(splited[0]), splited[1]
            first_right_state = int(splited[3])
            first_right_symbol = splited[4].replace('\n', '')
            if len(state_table[first_right_state]['guid_symbols']) == 0 and first_right_symbol not in terminals and first_right_symbol != eps:
                if isAllGuidSymbolsAdded(first_right_symbol):
                    all_guid_symbols = getAllGuidSymbolsBySymbol(first_right_symbol)
                    state_table[first_right_state]['guid_symbols'] = all_guid_symbols.copy()
                    state_table[left_state]['guid_symbols'] = state_table[first_right_state]['guid_symbols'].copy()

    for row1 in rows:
        if row1.split(' ')[-1].replace('\n', '') == eps:
            processEpsRule(row1)
    
    main_state_table = deepcopy(state_table)

# processEpsRule('12 ПараметрыФункции ::= 93 /eps')
# state_table[12]

In [8]:
count = 0
for state, v in state_table.items():
    if len(v['guid_symbols']) != 0:
        count += 1

# state_table
print(count)
print(rule - count) # 109 осталось заполнить

130
83


In [9]:
state_table

{1: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': [],
  'next': 71},
 2: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['int', 'void', 'bool'],
  'next': 72},
 3: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['int', 'void', 'bool'],
  'next': 75},
 4: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['int', 'void', 'bool'],
  'next': 78},
 5: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['int'],
  'next': 81},
 6: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['void'],
  'next': 82},
 7: {'error': 0,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['bool'],
  'next': 83},
 8: {'error': True,
  'stack': False,
  'return': False,
  'accept': False,
  'guid_symbols': ['('],
  'next': 84},
 9: {'error': 0,
  'stack

In [15]:
state_table[3]

{'error': 0,
 'stack': False,
 'return': False,
 'accept': False,
 'guid_symbols': ['int', 'void', 'bool'],
 'next': 75}

In [16]:
list(state_table[3].values())

[0, False, False, False, ['int', 'void', 'bool'], 75]

In [24]:
t = True
f = False
n = 4
diction = {0 : f, 1: t}

def generation2vec(d):
    n = len(d)
    a = [0] * n
    P = [a.copy()]
    while True:
        if a == [1] * n:
            break

        for i in range(n - 1, -1, -1):
            if a[i] == 0:
                idx = i
                break

        a[idx:] = [0] * (n - idx)
        a[idx] = 1
        P.append([a[i] for i in range(n)])
    
    for i in range(len(P)):
        for j in range(len(P[0])):
            P[i][j] = diction[P[i][j]]

    return P

for row in generation2vec([True, True, False, False]):
    print(f"{row} : {{}},")

[False, False, False, False] : {},
[False, False, False, True] : {},
[False, False, True, False] : {},
[False, False, True, True] : {},
[False, True, False, False] : {},
[False, True, False, True] : {},
[False, True, True, False] : {},
[False, True, True, True] : {},
[True, False, False, False] : {},
[True, False, False, True] : {},
[True, False, True, False] : {},
[True, False, True, True] : {},
[True, True, False, False] : {},
[True, True, False, True] : {},
[True, True, True, False] : {},
[True, True, True, True] : {},


In [45]:
class Stack:
    def __init__(self):
        self.stack = []

    def push(self, value):
        self.stack.append(value)
        
    def pop(self):
        try:
            # return self.stack.pop()
            return lambda: self.stack.pop()
        except IndexError:
            # return IndexError
            return lambda: IndexError

    def getLength(self):
        return len(self.stack)

stack2 = Stack()

stack2.push(1)
print(stack2.pop()())
try:
    a = stack2.pop()()
except IndexError:
    print("WOW")

1
WOW
